In [1]:
#https://svaderia.github.io/articles/downloading-and-unzipping-a-zipfile/
import requests
from urllib.request import urlopen
from tempfile import NamedTemporaryFile
import shutil
from shutil import unpack_archive

import rasterio as rio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.windows import Window
from rasterio.mask import mask  

import json

In [2]:
# download and unzip the dtm and dsm files based on URL
def download_unzip(url):

    with urlopen(url) as zipresp, NamedTemporaryFile() as tfile:
        tfile.write(zipresp.read())
        tfile.seek(0)
        filename = url.split('/')[-1].replace(".zip", "")
        file_path = f"/home/giodsubuntu/3D_house_Lidar/Files/{filename}"
        unpack_archive(tfile.name,file_path, format = 'zip')
        
    return filename

In [3]:
# Open the geotiff file in the previously downloaded folder
all_bounds ={} #where do I put ths to not keep on deleting my keys?

def open_geotiff(filename):
    file_path = f"/home/giodsubuntu/3D_house_Lidar/Files/{filename}/GeoTIFF/{filename}.tif"
    file_num = (file_path.split('/')[-1].replace(".tif", ""))[-2:]

    with rio.open(file_path) as geotiff:
        left, bottom, right, top = geotiff.bounds
        all_bounds[f"{file_num}"] = f"{left, bottom, right, top}"
        with open("/home/giodsubuntu/3D_house_Lidar/json/j_all_bounds.json", "a+") as f:
            json.dump(all_bounds, f)
            print(f"done{file_num}")
    return all_bounds

In [4]:
  #delete the folder

def zip_delete(filename):
    file_path = f"/home/giodsubuntu/3D_house_Lidar/Files/{filename}"
    shutil.rmtree(file_path, ignore_errors=True)
    return 

for num in range(0,9): 
    num +=1 
    url=f'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k0{num}.zip' 
    url_list.append(url) 
for num in range(9,43): 
    num +=1 
    url=f'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k{num}.zip' 
    url_list.append(url)

print(url_l)

In [5]:
url_list =['https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k34.zip', 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k35.zip', 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k36.zip', 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k37.zip', 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k38.zip', 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k39.zip', 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k40.zip', 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k41.zip', 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k42.zip', 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k43.zip']
for url in url_list:
    filename = download_unzip(url)
    
    boundaries = open_geotiff(filename)
    zip_delete(filename)

done34
done35
done36
done37
done38
done39
done40
done41
done42
done43


In [ ]:
print(all_bounds)


In [ ]:
#DHMVII_vdc_k31

#Do you REALLY need to unzip the 2nd file? The Geotiff is there!

#Nice to have: 
# better path for saving file
# handle file already downloaded (chg name?, abort?)